In [1]:
import numpy as np
import netCDF4
import matplotlib.pyplot as plot
import matplotlib as mpl
import wrf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import math
import metpy.calc as mpcalc

from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, vertcross, CoordPair
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy import config
from metpy.units import units

plot.switch_backend('agg')

In [2]:
def bearing(pointA, pointB):
    import math
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")
    #convert to radians
    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])
    #how far appart are the points in longitude
    diffLong = math.radians(pointB[1] - pointA[1])
    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))
    initial_bearing = math.atan2(x, y)
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360
    compass_bearing2 = abs(compass_bearing - 90.)
    return compass_bearing2

In [3]:
ncfile = Dataset('./wrfout_d01_2019-03-11_18:00:00')

In [4]:
lon = getvar(ncfile, "lon", meta=False)
lat = getvar(ncfile, "lat", meta=False)
Press = getvar(ncfile, "pressure", meta=False)
height = getvar(ncfile, "z")
UComp = getvar(ncfile, "ua", meta=False)
VComp = getvar(ncfile, "va", meta=False)
theta = getvar(ncfile, "theta", meta=False)

In [5]:
interp_level_800 = [800]

Height_800 = wrf.vinterp(ncfile, field=height, vert_coord="pressure", interp_levels=interp_level_800, extrapolate=True, field_type="ght")
U_800 = wrf.vinterp(ncfile, field=UComp, vert_coord="pressure", interp_levels=interp_level_800, extrapolate=True)
V_800 = wrf.vinterp(ncfile, field=VComp, vert_coord="pressure", interp_levels=interp_level_800, extrapolate=True)
Theta_800 = wrf.vinterp(ncfile, field=theta, vert_coord="pressure", interp_levels=interp_level_800, extrapolate=True, field_type="th")

U_800 = np.array(U_800)
V_800 = np.array(V_800)
Height_800 = np.squeeze(Height_800)
Vx_800 = np.squeeze(U_800)
Vy_800 = np.squeeze(V_800)

dx = 25000
dy = 25000

In [6]:
# Calculate total wind frontogenesis (in K/100km/3hr)
TW_fgen_800 = mpcalc.frontogenesis(Theta_800.data*units.K, U_800[::-1,:]*units.meter/units.second, V_800[::-1,:]*units.meter/units.second, dx*units.meter, dy*units.meter)*(1000*100*3600*3)
TW_fgen_800 = np.squeeze(TW_fgen_800)

In [7]:
lat1 = 40.
lon1 = -106.
lat2 = 40.
lon2 = -99.

start_pt = CoordPair(lat=lat1,lon=lon1)
end_pt = CoordPair(lat=lat2,lon=lon2)
geodetic = ccrs.Geodetic()
PC = ccrs.PlateCarree()
lon1_, lat1_ = PC.transform_point(lon1, lat1, geodetic)
lon2_, lat2_ = PC.transform_point(lon2, lat2, geodetic)


In [8]:
# Setting height contours
cint_800_h = 25 # Height interval every 50 m
cmin_800_h = np.min(Height_800) # Minimum height value
cmax_800_h = np.max(Height_800) # Maximum height value
clevs_800_h = np.arange(cmin_800_h, cmax_800_h, cint_800_h) # Height array to plot

cart_proj_800 = get_cartopy(height) # Plot projection

In [9]:
# Setting frontogensis contours
Fgen_levs = [-10,-9, -8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [10]:
# Plot
plot.rcParams['contour.negative_linestyle'] = 'solid'
fig = plot.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1, projection=cart_proj_800)
ax.set_extent([-115,-80,30,45])
ax.add_feature(cfeature.COASTLINE, edgecolor="gray", linewidth=2)
ax.add_feature(cfeature.BORDERS, edgecolor="gray", linewidth=2)
ax.add_feature(cfeature.STATES, edgecolor="gray", linewidth=1)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.ylocator = mticker.FixedLocator([20, 25, 30, 35, 40, 45, 50, 55])
gl.xlocator = mticker.FixedLocator([-60, -75, -90, -105, -120, -135])
gl.xlabels_top = True
gl.xlabels_bottom = True
gl.ylabels_left = True
gl.ylabels_right = True
gl.x_inline = False
gl.yinline = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
Fgen = plot.contourf(to_np(lon), to_np(lat), to_np(TW_fgen_800), Fgen_levs, cmap='PuOr', extend = 'both', transform=ccrs.PlateCarree())
cbar = plot.colorbar(Fgen, orientation='horizontal')
cbar.set_ticks([-10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
cbar.set_label('Frontogenesis (K per 100 km per 3 hr)', fontsize=15)
cntrs = plot.contour(to_np(lon), to_np(lat), to_np(Height_800), clevs_800_h, colors='k', linewidth=1, transform=ccrs.PlateCarree())
plot.clabel(cntrs, inline=1, fontsize=10, fmt='%i')
cl=plot.plot([lon1_, lon2_], [lat1_, lat2_], color='red', linewidth=3, transform=ccrs.PlateCarree())
plot.title('800 hPa Isobaric Level', fontsize=20)
plot.savefig('Fig.jpg')

/glade/work/ekoval/conda-envs/dynamics_class/lib/python3.7/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/glade/work/ekoval/conda-envs/dynamics_class/lib/python3.7/site-packages/cartopy/mpl/gridliner.py:463: UserWarning: The .xlabels_bottom attribute is deprecated. Please use .bottom_labels to toggle visibility instead.
  warnings.warn('The .xlabels_bottom attribute is deprecated. Please '
/glade/work/ekoval/conda-envs/dynamics_class/lib/python3.7/site-packages/cartopy/mpl/gridliner.py:475: UserWarning: The .ylabels_left attribute is deprecated. Please use .left_labels to toggle visibility instead.
  warnings.warn('The .ylabels_left attribute is deprecated. Please '
/glade/work/ekoval/conda-envs/dynamics_class/lib/python3.7/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is dep